## Sinusoidal 위치 인코딩

In [ ]:
import torch
import numpy as np

# 1. 환경 설정 및 하이퍼파라미터 정의
vocab_size = 50257
output_dim = 256
max_length = 4

# 2. 토큰 임베딩 층 (기존과 동일)
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

# 3. 데이터 로더 및 샘플 추출 (기존과 동일)
# dataloader = create_dataloader_v1(...) 
# inputs, targets = next(iter(dataloader))
# 예시를 위해 가상의 inputs 생성 (8개 배치, 4개 단어)
inputs = torch.randint(0, vocab_size, (8, max_length))

# 4. 토큰 임베딩 수행
token_embeddings = token_embedding_layer(inputs) # [8, 4, 256]

# 5. 삼각함수 기반 위치 인코딩 행렬 생성 (핵심 변경 부분)
def get_sinusoidal_encoding(seq_len, d_model):
    # 각 차원에 적용될 주파수(지수 부분) 계산
    # 10000^(2i/d_model)
    inv_freq = 1.0 / (10000 ** (torch.arange(0, d_model, 2).float() / d_model))
    
    # 위치 인덱스 (0, 1, 2, 3) 생성
    pos = torch.arange(seq_len).float().unsqueeze(1) # [seq_len, 1]
    
    # 사인/코사인 내부 값 계산
    # [seq_len, 1] * [1, d_model/2] -> [seq_len, d_model/2]
    sin_cos_args = pos * inv_freq
    
    # 최종 행렬 생성
    pe = torch.zeros(seq_len, d_model)
    pe[:, 0::2] = torch.sin(sin_cos_args) # 짝수 열은 Sine
    pe[:, 1::2] = torch.cos(sin_cos_args) # 홀수 열은 Cosine
    return pe

# 위치 인코딩 행렬 생성
pos_encodings = get_sinusoidal_encoding(max_length, output_dim)

# 6. 생성된 위치 인코딩 확인
print(f"Position Encoding Shape: {pos_encodings.shape}") # [4, 256]
print("Position Encoding Values (First 2 rows):\n", pos_encodings[:2, :10])

# 7. 최종 입력 임베딩 생성 (토큰 정보 + 위치 정보)
# 학습되는 파라미터가 아니므로 가중치 업데이트가 필요 없음을 명시하기 위해 .detach()를 쓰기도 합니다.
input_embeddings = token_embeddings + pos_encodings
print(f"Final Input Shape: {input_embeddings.shape}") # [8, 4, 256]